# Aula 5 Funçoes e estruturas

## Imports

In [84]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim

## Loads

In [56]:
df=pd.read_csv('datasets/kc_house_data.csv')

## Data dimensions

In [57]:
print('Number of rows:{}'.format(df.shape[0]))
print('Number of cols:{}'.format(df.shape[1]))

Number of rows:21613
Number of cols:21


## Convert data

In [58]:
df['date']=pd.to_datetime(df['date'])
df.dtypes

id                        int64
date             datetime64[ns]
price                   float64
bedrooms                  int64
bathrooms               float64
sqft_living               int64
sqft_lot                  int64
floors                  float64
waterfront                int64
view                      int64
condition                 int64
grade                     int64
sqft_above                int64
sqft_basement             int64
yr_built                  int64
yr_renovated              int64
zipcode                   int64
lat                     float64
long                    float64
sqft_living15             int64
sqft_lot15                int64
dtype: object

## Statistics descriptions

In [66]:
num_attributes=df.select_dtypes(include=('int64','float64'))

In [67]:
print('Number of rows:{}'.format(num_attributes.shape[1]))
num_attributes.dtypes

Number of rows:20


id                 int64
price            float64
bedrooms           int64
bathrooms        float64
sqft_living        int64
sqft_lot           int64
floors           float64
waterfront         int64
view               int64
condition          int64
grade              int64
sqft_above         int64
sqft_basement      int64
yr_built           int64
yr_renovated       int64
zipcode            int64
lat              float64
long             float64
sqft_living15      int64
sqft_lot15         int64
dtype: object

In [68]:
pd.set_option('display.float_format', lambda x: f'%.{len(str(x%1))-2}f' % x)

media=pd.DataFrame(num_attributes.apply(np.mean, axis=0))
mediana=pd.DataFrame(num_attributes.apply(np.median,axis=0))

maximo=pd.DataFrame(num_attributes.apply(np.max,axis=0))
minimo=pd.DataFrame(num_attributes.apply(np.min,axis=0))
std=pd.DataFrame(num_attributes.apply(np.std,axis=0))

df1=pd.concat([maximo,minimo,media,mediana,std], axis=1).reset_index()
df1.columns=['attributes','maximo','minimo','media','mediana','std']

In [69]:
df1

,attributes,maximo,minimo,media,mediana,std
0,id,9900000190.0,1000102.0,4580301520.8649883270263672,3904930410.0,2876499023.4277825355529785
1,price,7700000.0,75000.0,540088.14176652941387147,450000.0,367118.7031813723151572
2,bedrooms,33.0,0.0,3.3708416230972098,3.0,0.9300403146391111
3,bathrooms,8.0,0.0,2.1147573219821405,2.25,0.7701453398840542
4,sqft_living,13540.0,290.0,2079.8997362698373763,1910.0,918.41964938288231224
5,sqft_lot,1651359.0,520.0,15106.9675658168689552,7618.0,41419.5532726276796893
6,floors,3.5,1.0,1.49430898070605656,1.5,0.5399764027733692
7,waterfront,1.0,0.0,0.007541757275713691,0.0,0.08651519619643655
8,view,4.0,0.0,0.23430342849211122,0.0,0.7662998409056494
9,condition,5.0,1.0,3.40942951001711947,3.0,0.6507279917561631


### Crie uma nova coluna chamada 'dormitory_type'
- se o valor dacoluna 'bedrooms' for igual a 1 = studio
- se o valor dacoluna 'bedrooms' for igual a 2 = apartment
- se o valor dacoluna 'bedrooms' for igual a 3 = house

In [81]:
df['dormitory_type']='NA'

for i in range(len(df)):
    if df.loc[i,'bedrooms']==1:
        df.loc[i,'dormitory_type']='studio'
    elif  df.loc[i,'bedrooms']==2:
        df.loc[i,'dormitory_type']='apartment'
    else:
        df.loc[i,'dormitory_type']='house'
    

### Qual a quantidade de imóveis por nível?
- Nivel 0: Preço entre R$ 0.00 e R$ 321.950
- Nivel 1: Preço entre R$ 321.950 e R$ 450.000
- Nivel 2: Preço entre R$ 450.000 e R$ 645.000
- Nivel 3: Preço entre R$ Acima de R$ 645.000

In [82]:
df['level']='NA'

for i in range(len(df)):
    if (df.loc[i,'price']>=0)&(df.loc[i,'price']>321.950):
        df.loc[i,'level']=1
    elif (df.loc[i,'price']>=321.950)&(df.loc[i,'price']>450.000):
        df.loc[i,'level']=2
    elif (df.loc[i,'price']>=450.000)&(df.loc[i,'price']>645.000):
        df.loc[i,'level']=3
    else:
        df.loc[i,'level']=4
    

In [83]:
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,dormitory_type,level
0,7129300520,2014-10-13,221900.0,3,1.0,1180,5650,1.0,0,0,...,0,1955,0,98178,47.5112000000000023,-122.257000000000005,1340,5650,house,1
1,6414100192,2014-12-09,538000.0,3,2.25,2570,7242,2.0,0,0,...,400,1951,1991,98125,47.7209999999999965,-122.3190000000000026,1690,7639,house,1
2,5631500400,2015-02-25,180000.0,2,1.0,770,10000,1.0,0,0,...,0,1933,0,98028,47.7379000000000033,-122.2330000000000041,2720,8062,apartment,1
3,2487200875,2014-12-09,604000.0,4,3.0,1960,5000,1.0,0,0,...,910,1965,0,98136,47.5208000000000013,-122.3930000000000007,1360,5000,house,1
4,1954400510,2015-02-18,510000.0,3,2.0,1680,8080,1.0,0,0,...,0,1987,0,98074,47.6167999999999978,-122.0450000000000017,1800,7503,house,1
